In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# fetch page content
url = "https://www.animeboston.com/schedule/index/2024"
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

# find all tables
tables = soup.select("table.schedule_table")

# container for events
events = []



In [2]:
soup

<!DOCTYPE HTML>

<html>
<head>
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-38TMR0STT6"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'G-38TMR0STT6');
</script>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>Programming Schedule</title>
<link href="/images/favicon/57x57.png" rel="apple-touch-icon" type="image/png"/>
<link href="/images/favicon/76x76.png" rel="apple-touch-icon" sizes="76x76" type="image/png"/>
<link href="/images/favicon/120x120.png" rel="apple-touch-icon" sizes="120x120" type="image/png"/>
<link href="/images/favicon/152x152.png" rel="apple-touch-icon" sizes="152x152" type="image/png"/>
<link href="/images/favicon/180x180.png" rel="apple-touch-icon" sizes="180x180" type="image/png"/>
<link href="https://s3.amazonaws.com/www-ab/img/favico192.png" rel="icon" sizes="192x192" type="image/png"/>
<link href=

In [6]:
# extract legend
legend_items = soup.select("div.schedule-legend label.schedule-category-label")

# build color map
category_colors = {
    label.text.strip(): label.get("style").split("background-color:")[1].strip()
    for label in legend_items if "background-color:" in label.get("style", "")
}

In [7]:
category_colors

{'18+ Cosplay': '#AA99FF',
 '18+ Event': '#CC9966',
 '18+ Fan Creations': '#669966',
 '18+ Fan Panel': '#669900',
 '18+ Featured Panel': '#66CC00',
 '18+ Gameshow': '#9900FF',
 '18+ Guest Panel': '#669999',
 '18+ Video': '#F7921B',
 '21+ Event': '#FF0000',
 'Ball': '#CC9900',
 'Concerts': '#996633',
 'Cosplay Games': '#EEAEEE',
 'Event': '#E8D37E',
 'Fan Creations': '#CCFF66',
 'Fan Panel': '#00FF00',
 'Featured Artist': '#CCFFCC',
 'Featured Panel': '#00FF99',
 'Gameshow': '#72B7ED',
 'Guest Panel': '#00FFFF',
 'ID Check': '#999999',
 'Idol Events': '#FF99CC',
 'Industry Panel': '#3399FF',
 'Jam Zone': '#CC0000',
 'Karaoke': '#9933FF',
 'Libraries & Education': '#CCFF00',
 'Lolita & J-fashion': '#CC6699',
 'Maid Cafe': '#CCCCFF',
 'Premiere Video': '#FF9966',
 'Room Clear': '#CCCCCC',
 'Seating': '#CCCCCC',
 'Social Gatherings': '#FF3366',
 'Tabletop Gaming': '#CC0066',
 'Video': '#66A3D2',
 'Workshop': '#F7931E'}

In [69]:
room_headers = soup.select("table.schedule-table tr")[1].select("th.schedule-room")
rooms = [room.get_text(strip=True) for room in room_headers]
rooms

['Auditorium Events',
 'Ballroom A',
 'Ballroom B',
 'Fan Creations 312',
 'Maid Cafe',
 'Panel 202',
 'Panel 207',
 'Panel 208',
 'Panel 302',
 'Panel 309',
 'Panel 310',
 'Video 210',
 'Video 306',
 'Kings',
 'Grand Ballroom',
 'Panel Constitution',
 'Panel Gardner',
 'Panel The Fens',
 'Republic Ballroom',
 'RPG Riverway',
 'Video Hampton',
 'Workshop Fairfax']

In [83]:
tables = pd.read_html(url)  # This reads the first two rows as column headers
display(tables)

[   Unnamed: 0_level_0              Hynes                             \
    Unnamed: 0_level_1  Auditorium Events     Ballroom A  Ballroom B   
 0             8:00 am                NaN            NaN         NaN   
 1             8:15 am                NaN            NaN         NaN   
 2             8:30 am                NaN            NaN         NaN   
 3             8:45 am                NaN            NaN         NaN   
 4             9:00 am                NaN            NaN         NaN   
 ..                ...                ...            ...         ...   
 69            1:15 am                NaN  Slumber Party         NaN   
 70            1:30 am                NaN            NaN         NaN   
 71            1:45 am                NaN            NaN         NaN   
 72                NaN  Auditorium Events     Ballroom A  Ballroom B   
 73                NaN              Hynes          Hynes       Hynes   
 
                                                              

In [87]:
def preprocess_tables(tables, ii, FirstColumnName='TimeSlot'):
    tables[ii].dropna(subset=[tables[ii].columns[0]], inplace=True)
    tables[ii] = tables[ii].iloc[:, :-1].copy()
    tables[ii].columns = [FirstColumnName] + tables[ii].columns[1:].tolist()
    return tables[ii]

In [96]:
df = preprocess_tables(tables,0)
df

,TimeSlot,"(Hynes, Auditorium Events)","(Hynes, Ballroom A)","(Hynes, Ballroom B)","(Hynes, Fan Creations 312)","(Hynes, Maid Cafe)","(Hynes, Panel 202)","(Hynes, Panel 207)","(Hynes, Panel 208)","(Hynes, Panel 302)","(Hynes, Panel 309)","(Hynes, Panel 310)","(Hynes, Video 210)","(Hynes, Video 306)","(Kings, Kings)","(Sheraton, Grand Ballroom)","(Sheraton, Panel Constitution)","(Sheraton, Panel Gardner)","(Sheraton, Panel The Fens)"
0,8:00 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lupin the 3rd: The Castle of Cagliost ro,Dual! Parallel Trouble Adventur e,NaN,NaN,NaN,NaN,NaN
1,8:15 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lupin the 3rd: The Castle of Cagliost ro,Dual! Parallel Trouble Adventur e,NaN,NaN,NaN,NaN,NaN
2,8:30 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lupin the 3rd: The Castle of Cagliost ro,Dual! Parallel Trouble Adventur e,NaN,NaN,NaN,NaN,NaN
3,8:45 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lupin the 3rd: The Castle of Cagliost ro,Dual! Parallel Trouble Adventur e,NaN,NaN,NaN,NaN,NaN
4,9:00 am,NaN,NaN,NaN,AMV Genkis,NaN,NaN,NaN,NaN,NaN,NaN,There's a Live Action Version! ? An Intro to A...,Lupin the 3rd: The Castle of Cagliost ro,Dual! Parallel Trouble Adventur e,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,12:45 am,NaN,Slumber Party,NaN,Fanservi ce & Ecchi AMVs (18+),NaN,NaN,NaN,NaN,Taskmast er (18+),ID Check Seating (18+),Masculin ity and Martyrdo m in Yaoi and BL (18+),NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,1:00 am,NaN,Slumber Party,NaN,Horror AMVs (18+),NaN,NaN,NaN,NaN,Taskmast er (18+),"Anime Statisti cs Hentai Edition - Numbers, Fo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,1:15 am,NaN,Slumber Party,NaN,Horror AMVs (18+),NaN,NaN,NaN,NaN,Taskmast er (18+),"Anime Statisti cs Hentai Edition - Numbers, Fo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,1:30 am,NaN,NaN,NaN,Horror AMVs (18+),NaN,NaN,NaN,NaN,Taskmast er (18+),"Anime Statisti cs Hentai Edition - Numbers, Fo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
# add original row order as a helper column
df["RowOrder"] = df.index

# melt while preserving row order
long_df = df.melt(id_vars=["TimeSlot", "RowOrder"], 
                  var_name="Room", 
                  value_name="Event")

# drop rows without an event
events_df = long_df.dropna(subset=["Event"]).sort_values("RowOrder").drop(columns="RowOrder").reset_index(drop=True)
events_df

,TimeSlot,Room,Event
0,8:00 am,"(Hynes, Video 210)",Lupin the 3rd: The Castle of Cagliost ro
1,8:00 am,"(Hynes, Video 306)",Dual! Parallel Trouble Adventur e
2,8:15 am,"(Hynes, Video 306)",Dual! Parallel Trouble Adventur e
3,8:15 am,"(Hynes, Video 210)",Lupin the 3rd: The Castle of Cagliost ro
4,8:30 am,"(Hynes, Video 210)",Lupin the 3rd: The Castle of Cagliost ro
...,...,...,...
711,1:30 am,"(Hynes, Fan Creations 312)",Horror AMVs (18+)
712,1:30 am,"(Hynes, Panel 302)",Taskmast er (18+)
713,1:45 am,"(Hynes, Panel 309)","Anime Statisti cs Hentai Edition - Numbers, Fo..."
714,1:45 am,"(Hynes, Fan Creations 312)",Horror AMVs (18+)


In [125]:
# group by Event and aggregate TimeSlot into a set
df_grouped = events_df.groupby(['Event','Room'])['TimeSlot'].agg(list).reset_index()
df_grouped.to_csv('test.csv', index=False, encoding='utf-8-sig')
df_grouped

,Event,Room,TimeSlot
0,"""Your Mom vs. the Noobs"" aka Otaku Mad Libs","(Sheraton, Panel Gardner)","[5:30 pm, 5:45 pm, 6:00 pm, 6:15 pm]"
1,50% Off,"(Hynes, Fan Creations 312)","[8:45 pm, 9:00 pm, 9:15 pm, 9:30 pm, 9:45 pm, ..."
2,A Brief History of Anime at the Movies,"(Hynes, Panel 207)","[4:30 pm, 4:45 pm, 5:00 pm, 5:15 pm]"
3,A Certain Magical Index,"(Hynes, Video 210)","[10:00 am, 10:15 am, 10:30 am, 10:45 am, 11:00..."
4,A Taste Of Reality,"(Hynes, Fan Creations 312)","[10:45 pm, 11:00 pm, 11:15 pm, 11:30 pm]"
...,...,...,...
132,Why People Love Japanese Snacks,"(Hynes, Panel 202)","[9:30 am, 9:45 am, 10:00 am, 10:15 am]"
133,Working in the JAV industry to anime voice act...,"(Hynes, Panel 208)","[7:00 pm, 7:15 pm, 7:30 pm, 7:45 pm]"
134,"Worldwea ving: Characte r Design, Clothing His...","(Hynes, Panel 309)","[11:00 am, 11:15 am, 11:30 am, 11:45 am]"
135,"Yoko Taro-ver se: Sex, Death, and Violence (18+)","(Hynes, Panel 207)","[6:00 pm, 6:15 pm, 6:30 pm, 6:45 pm]"


In [121]:
df_grouped[df_grouped['Event']=='Maid Café']

,Event,Room,TimeSlot
86,Maid Café,"(Hynes, Maid Cafe)","[12:00 pm, 12:15 pm, 12:30 pm, 12:45 pm, 1:00 ..."


In [122]:
import pandas as pd

def split_event_to_subevents(df, event_col, time_col, room_col, chunk_size=3, target_event=None):
    """
    Splits a specific event into subevents by grouping its time slots into chunks.
    
    Parameters:
    - df: The DataFrame containing the events.
    - event_col: The name of the column containing event names.
    - time_col: The name of the column containing time slots.
    - room_col: The name of the column containing room information.
    - chunk_size: The number of time slots per subevent (default 3).
    - target_event: The specific event to split (e.g., 'Maid Café'). If None, all events are processed.
    
    Returns:
    - A DataFrame with subevents, grouped by event and room.
    """
    # If target_event is provided, filter only that event
    if target_event:
        df = df[df[event_col] == target_event]

    # Explode the TimeSlot column and sort by Room and TimeSlot
    exploded_df = (df.explode(time_col)
                   .sort_values(by=[room_col, time_col])
                   .reset_index(drop=True))

    # Create subevent labels based on chunking logic
    exploded_df['Subevent'] = exploded_df.groupby(room_col).cumcount() // chunk_size
    exploded_df['Subevent'] = exploded_df['Subevent'].apply(lambda x: f"{df[event_col].iloc[0]} {chr(65 + x)}")

    # Group by subevent and room, aggregate time slots into lists
    return exploded_df.groupby(['Subevent', room_col])[time_col].agg(list).reset_index()

# Example usage
# Assuming 'Event', 'TimeSlot', 'Room' are the column names in your DataFrame
split_event_to_subevents(df_grouped, 'Event', 'TimeSlot', 'Room', chunk_size=3, target_event="Maid Café")


,Subevent,Room,TimeSlot
0,Maid Café A,"(Hynes, Maid Cafe)","[12:00 pm, 12:15 pm, 12:30 pm]"
1,Maid Café B,"(Hynes, Maid Cafe)","[12:45 pm, 1:00 pm, 1:15 pm]"
2,Maid Café C,"(Hynes, Maid Cafe)","[1:30 pm, 1:45 pm, 2:00 pm]"
3,Maid Café D,"(Hynes, Maid Cafe)","[2:15 pm, 2:30 pm, 2:45 pm]"
4,Maid Café E,"(Hynes, Maid Cafe)","[3:00 pm, 3:15 pm, 3:30 pm]"
5,Maid Café F,"(Hynes, Maid Cafe)","[3:45 pm, 4:00 pm, 4:15 pm]"
6,Maid Café G,"(Hynes, Maid Cafe)","[4:30 pm, 4:45 pm, 5:00 pm]"
7,Maid Café H,"(Hynes, Maid Cafe)","[5:15 pm, 5:30 pm, 5:45 pm]"


### Notes

- Maid Cafe subevents are optional, but crucial to capture when repeating events
- So what is needed next is to have the event and timeslot taken out and scored with utility, combined with metadata such as the category number or whatnot.

In [85]:
preprocess_tables(tables,1)

,TimeSlot,"(Hynes, Auditorium Events)","(Hynes, Ballroom A)","(Hynes, Ballroom B)","(Hynes, Fan Creations 312)","(Hynes, Karaoke 313)","(Hynes, Maid Cafe)","(Hynes, Panel 202)","(Hynes, Panel 207)","(Hynes, Panel 208)",...,"(Hynes, Video 306)","(Kings, Kings)","(Sheraton, Grand Ballroom)","(Sheraton, Panel Constitution)","(Sheraton, Panel Gardner)","(Sheraton, Panel The Fens)","(Sheraton, Republic Ballroom)","(Sheraton, RPG Riverway)","(Sheraton, Video Hampton)","(Sheraton, Workshop Fairfax)"
0,8:00 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,RWBY (PG),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tales of Zestria the X,NaN
1,8:15 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,RWBY (PG),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tales of Zestria the X,NaN
2,8:30 am,NaN,NaN,Seating,NaN,NaN,NaN,NaN,NaN,NaN,...,RWBY (PG),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tales of Zestria the X,NaN
3,8:45 am,NaN,NaN,Seating,NaN,NaN,NaN,NaN,NaN,NaN,...,RWBY (PG),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tales of Zestria the X,NaN
4,9:00 am,NaN,NaN,AMV Contest,Enhancin g Your AMV Experien ce,NaN,NaN,NaN,NaN,NaN,...,RWBY (PG),NaN,NaN,NaN,NaN,NaN,narcolep syDriver and Friends !!,Tabletop Session 1 - A Strange New World,Tales of Zestria the X,Let's $ew! Eggplant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,12:45 am,NaN,Slumber Party,Hentai AMV Extravag anza! (18+),Comedy on the Edge (18+),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,1:00 am,NaN,Slumber Party,Hentai AMV Extravag anza! (18+),(18+) AMV Freeplay (18+),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,1:15 am,NaN,Slumber Party,Hentai AMV Extravag anza! (18+),(18+) AMV Freeplay (18+),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,1:30 am,NaN,NaN,Hentai AMV Extravag anza! (18+),(18+) AMV Freeplay (18+),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
preprocess_tables(tables,2)

,TimeSlot,"(Hynes, Auditorium Events)","(Hynes, Ballroom A)","(Hynes, Ballroom B)","(Hynes, Fan Creations 312)","(Hynes, Maid Cafe)","(Hynes, Panel 202)","(Hynes, Panel 207)","(Hynes, Panel 208)","(Hynes, Panel 302)",...,"(Hynes, Video 210)","(Hynes, Video 306)","(Sheraton, Grand Ballroom)","(Sheraton, Panel Constitution)","(Sheraton, Panel Gardner)","(Sheraton, Panel The Fens)","(Sheraton, Republic Ballroom)","(Sheraton, RPG Riverway)","(Sheraton, Video Hampton)","(Sheraton, Workshop Fairfax)"
0,8:00 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,One Punch Man,Ascendan ce of a bookworm,NaN,NaN,NaN,NaN,NaN,NaN,Applesee d 1988 OVA (TVMA),NaN
1,8:15 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,One Punch Man,Ascendan ce of a bookworm,NaN,NaN,NaN,NaN,NaN,NaN,Applesee d 1988 OVA (TVMA),NaN
2,8:30 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,One Punch Man,Ascendan ce of a bookworm,NaN,NaN,NaN,NaN,NaN,NaN,Applesee d 1988 OVA (TVMA),NaN
3,8:45 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,One Punch Man,Ascendan ce of a bookworm,NaN,NaN,NaN,NaN,NaN,NaN,Applesee d 1988 OVA (TVMA),NaN
4,9:00 am,NaN,NaN,NaN,AMV Contest Overflow,NaN,NaN,NaN,NaN,NaN,...,One Punch Man,Ascendan ce of a bookworm,NaN,A Puppet Without a Ghost: The Evolutio n of Cy...,NaN,NaN,NaN,NaN,Applesee d 1988 OVA (TVMA),NaN
5,9:15 am,NaN,NaN,NaN,AMV Contest Overflow,NaN,NaN,NaN,NaN,NaN,...,One Punch Man,Ascendan ce of a bookworm,NaN,A Puppet Without a Ghost: The Evolutio n of Cy...,NaN,NaN,NaN,NaN,Applesee d 1988 OVA (TVMA),NaN
6,9:30 am,NaN,NaN,NaN,AMV Contest Overflow,NaN,NaN,Embellis hments Galore: A primer on embroide r...,Worldbui lding 101,NaN,...,One Punch Man,Ascendan ce of a bookworm,Charity Auction,A Puppet Without a Ghost: The Evolutio n of Cy...,NaN,NaN,NaN,NaN,Applesee d 1988 OVA (TVMA),How to Frankens tein a Pattern for Cosplay
7,9:45 am,NaN,NaN,NaN,AMV Contest Overflow,NaN,NaN,Embellis hments Galore: A primer on embroide r...,Worldbui lding 101,NaN,...,One Punch Man,Ascendan ce of a bookworm,Charity Auction,A Puppet Without a Ghost: The Evolutio n of Cy...,NaN,NaN,NaN,NaN,Applesee d 1988 OVA (TVMA),How to Frankens tein a Pattern for Cosplay
8,10:00 am,NaN,NaN,NaN,AMV Contest Overflow,NaN,Origami: The Art of Paperfol ding,Embellis hments Galore: A primer on embroide r...,Worldbui lding 101,Anime Theme Bingo,...,"Goodbye, Don Glees (PG-13)",Fire Force (PG-13),Charity Auction,NaN,NaN,NaN,NaN,Tabletop Session 3 - Dancing With the Demon Lord,Applesee d 1988 OVA (TVMA),How to Frankens tein a Pattern for Cosplay
9,10:15 am,NaN,NaN,NaN,AMV Contest Overflow,NaN,Origami: The Art of Paperfol ding,Embellis hments Galore: A primer on embroide r...,Worldbui lding 101,Anime Theme Bingo,...,"Goodbye, Don Glees (PG-13)",Fire Force (PG-13),Charity Auction,NaN,NaN,NaN,NaN,Tabletop Session 3 - Dancing With the Demon Lord,Applesee d 1988 OVA (TVMA),How to Frankens tein a Pattern for Cosplay


In [54]:
# Rename the columns
tables[0].columns = ['Location', 'Room'] + tables[0].columns[2:].tolist()

# Now you can drop rows with NaN values in the first column
tables[0].dropna(subset=['Location'], inplace=True)

tables[0]

,Location,Room,"(Hynes, Ballroom A)","(Hynes, Ballroom B)","(Hynes, Fan Creations 312)","(Hynes, Maid Cafe)","(Hynes, Panel 202)","(Hynes, Panel 207)","(Hynes, Panel 208)","(Hynes, Panel 302)",...,"(Kings, Kings)","(Sheraton, Grand Ballroom)","(Sheraton, Panel Constitution)","(Sheraton, Panel Gardner)","(Sheraton, Panel The Fens)","(Sheraton, Republic Ballroom)","(Sheraton, RPG Riverway)","(Sheraton, Video Hampton)","(Sheraton, Workshop Fairfax)","(Unnamed: 23_level_0, Unnamed: 23_level_1)"
0,8:00 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Soul Eater,NaN,8:00 am
1,8:15 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Soul Eater,NaN,8:15 am
2,8:30 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Soul Eater,NaN,8:30 am
3,8:45 am,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Soul Eater,NaN,8:45 am
4,9:00 am,NaN,NaN,NaN,AMV Genkis,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Tabletop Session 1 - A Strange New World,Soul Eater,Let's $ew! Ditto!,9:00 am
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,12:45 am,NaN,Slumber Party,NaN,Fanservi ce & Ecchi AMVs (18+),NaN,NaN,NaN,NaN,Taskmast er (18+),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:45 am
68,1:00 am,NaN,Slumber Party,NaN,Horror AMVs (18+),NaN,NaN,NaN,NaN,Taskmast er (18+),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:00 am
69,1:15 am,NaN,Slumber Party,NaN,Horror AMVs (18+),NaN,NaN,NaN,NaN,Taskmast er (18+),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:15 am
70,1:30 am,NaN,NaN,NaN,Horror AMVs (18+),NaN,NaN,NaN,NaN,Taskmast er (18+),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1:30 am
